In [96]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [97]:
# I do not know where this came from
# Preprocess step because the imagenet training images used for the Xception was of size 224x224
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [98]:
# Getting the path to the data
from pathlib import Path
import os

path = Path(os.getcwd())
path

PosixPath('/home/ryuparish/Code/HOML/ml/Butterfly')

In [99]:
# Getting all the paths for each image of clothing in the dataset
# os.walk returns the current walking directory, the subdirectories within it, and then finally the files within it.
for name, subdirs, files in os.walk(path):
    # The name variable keeps gettin longer as you get further away from the path folder
    # aclImdb, aclImdb/test, aclImdb/test/neg ... 
    indent = len(Path(name).parts) - len(path.parts)
    # os.sep adds a "/" if necessary 
    print("    " * indent + Path(name).parts[-1] + os.sep)
    for index, filename in enumerate(sorted(files)):
        # If we have printed three items already, just go ahead and print and ellipsis and move on
        if index == 3:
            print("    " * (indent + 1) + "...")
            break
        print("    " * (indent + 1) + filename)

Butterfly/
    Transfer Learning Color Clothing.ipynb
    red_shoes/
        red_shoes_1.jpg
        red_shoes_10.jpg
        red_shoes_100.jpg
        ...
    white_dress/
        white_dress_1.jpg
        white_dress_10.jpg
        white_dress_100.jpg
        ...
    silver_shoes/
        silver_shoes_000001.jpg
        silver_shoes_000002.jpg
        silver_shoes_000003.jpg
        ...
    white_suit/
        white_suit_000001.jpg
        white_suit_000002.jpg
        white_suit_000003.jpg
        ...
    blue_shorts/
        blue_shorts_1.jpg
        blue_shorts_10.jpg
        blue_shorts_100.jpg
        ...
    white_shoes/
        white_shoes_1.jpg
        white_shoes_10.jpg
        white_shoes_100.jpg
        ...
    blue_shoes/
        blue_shoes_1.jpg
        blue_shoes_10.jpg
        blue_shoes_100.jpg
        ...
    blue_shirt/
        blue_shirt_1.jpg
        blue_shirt_10.jpg
        blue_shirt_100.jpg
        ...
    black_dress/
        black_dress_1.jpg
        black_d

In [100]:
# Function to get all the image paths and glob them into one large list and then convert the PosixPath's into
# strings
def image_paths(dirpath):
    # PosixPath.glob will get all the path names of a specific pattern and return all of them in a list
    return [str(path) for path in dirpath.glob("*.jpg")]

In [101]:
# Getting the paths to all the images
clothing_dataset = image_paths(path/"data")

In [102]:
# Making an encoder to store all the encoded versions of the clothing classes
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

# All classes
clothing_table = [
    "black_dress",
    "black_pants",
    "black_shirt",
    "black_shoes",
    "black_shorts",
    "black_suit",
    "blue_dress",
    "blue_pants",
    "blue_shirt",
    "blue_shoes",
    "blue_shorts",
    "brown_hoodie",
    "brown_pants",
    "brown_shoes",
    "green_pants",
    "green_shirt",
    "green_shoes",
    "green_shorts",
    "green_suit",
    "pink_hoodie",
    "pink_pants",
    "pink_skirt",
    "red_dress",
    "red_hoodie",
    "red_pants",
    "red_shirt",
    "red_shoes",
    "silver_shoes",
    "silver_skirt",
    "white_dress",
    "white_pants",
    "white_shoes",
    "white_shorts",
    "white_suit",
    "yellow_dress",
    "yellow_shorts",
    "yellow_skirt"
]

encoder.fit(clothing_table)
print(encoder.classes_)
print("This many classes:", len(encoder.classes_))

['black_dress' 'black_pants' 'black_shirt' 'black_shoes' 'black_shorts'
 'black_suit' 'blue_dress' 'blue_pants' 'blue_shirt' 'blue_shoes'
 'blue_shorts' 'brown_hoodie' 'brown_pants' 'brown_shoes' 'green_pants'
 'green_shirt' 'green_shoes' 'green_shorts' 'green_suit' 'pink_hoodie'
 'pink_pants' 'pink_skirt' 'red_dress' 'red_hoodie' 'red_pants'
 'red_shirt' 'red_shoes' 'silver_shoes' 'silver_skirt' 'white_dress'
 'white_pants' 'white_shoes' 'white_shorts' 'white_suit' 'yellow_dress'
 'yellow_shorts' 'yellow_skirt']
This many classes: 37


In [103]:
# For labelling each image
def label_image(filename):
    # Getting the last file instead of the entire file path
    depth_of_file = filename.count("/")
    split_filepath = filename.split("/", depth_of_file)
    real_filename = split_filepath[-1]\
    
    label = real_filename[0:real_filename.find("_", 7)]
    # Returning the image and the encoded label
    return filename, encoder.transform([label])[0]

# For showing the image
def show(image, label):
    plt.figure()
    plt.imshow(image)
    plt.title(label)
    plt.axis('off')

In [104]:
# Labelling the images
clothing_dataset = map(label_image, clothing_dataset)
clothing_dataset = list(clothing_dataset)

In [152]:
# Shuffling the dataset and splitting it up into train, validation, and test sets
np.random.shuffle(clothing_dataset)
clothing_dataset = np.asarray(clothing_dataset)
train = clothing_dataset[:11000]
valid = clothing_dataset[11000:14000]
test = clothing_dataset[14000:16171]

In [153]:
# Splitting into instances and labels
x_train = train[:,0]
y_train = train[:,1].astype(np.uint8)
x_valid = valid[:,0]
y_valid = valid[:,1].astype(np.uint8)
x_test = test[:,0]
y_test = test[:,1].astype(np.uint8)

In [154]:
# Image parsing and then attaching a label based on the name of the file
# Original
def parse_image(filepath, label):
    # Getting the image from the filepath
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    
    # Returning the image and the encoded label
    return image, label

In [158]:
# Image parsing and then attaching a label based on the name of the file
# Original
def parse_image(filepath, label):
    # Getting the image from the filepath
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    
    # Returning the image and the encoded label
    return image, label

In [162]:
# DEBUGGING
# Mapping each of the filepaths to be images


In [151]:
# Converting the instances to tf datasets
training_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
testing_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

ValueError: Attempt to convert a value (<map object at 0x7ff27c54f400>) with an unsupported type (<class 'map'>) to a Tensor.

In [130]:
# Turning the filepaths of images to their rightful images
training_dataset.map(parse_image).shuffle(1000).batch(32).prefetch(1)
valid_dataset.map(parse_image).shuffle(1000).batch(32).prefetch(1)
testing_dataset.map(parse_image)

<MapDataset shapes: ((224, 224, 3), ()), types: (tf.float32, tf.uint8)>

In [135]:
for something in training_dataset:
    print(something[0])
    break

tf.Tensor(b'/home/ryuparish/Code/HOML/ml/Butterfly/data/blue_dress_330.jpg', shape=(), dtype=string)


In [139]:
# Building the top of the model
n_classes = 37
base_model = tf.keras.applications.efficientnet.EfficientNetB0(weights="imagenet",
include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [140]:
# Freezing the weights so I don't ruin the good lower layers
for layer in base_model.layers:
    layer.trainable = False

In [141]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="nadam", metrics=["accuracy"])
model.fit(training_dataset, epochs=5, validation_data=valid_dataset)

Epoch 1/5


ValueError: in user code:

    /home/ryuparish/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /home/ryuparish/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ryuparish/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ryuparish/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ryuparish/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ryuparish/.local/lib/python3.8/site-packages/tensorflow/py